In [1]:
import os
import argparse
import json
import numpy as np
import torch
import torch.nn as nn

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import umap                        

from utils.util import find_max_epoch, print_size, training_loss, calc_diffusion_hyperparams
from utils.util import get_mask_mnr, get_mask_bm, get_mask_rm

from imputers.DiffWaveImputer import DiffWaveImputer
from imputers.SSSDSAImputer import SSSDSAImputer
from imputers.SSSDS4Imputer import SSSDS4Imputer


CUDA extension for cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%
Falling back on slow Cauchy kernel. Install at least one of pykeops or the CUDA extension for efficiency.


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('-c', '--config', type=str, default='./config/config_SSSDS4-sp500.json')  
# parser.add_argument('-c', '--config', type=str, default='./config/config_DiffWave-sp500.json') 
                                                                                                                                                                                                                               
args = parser.parse_args(args=[])

with open(args.config) as f:
    data = f.read()

config = json.loads(data)

In [4]:
train_config = config["train_config"]  # training parameters

global trainset_config
trainset_config = config["trainset_config"]  # to load trainset

global diffusion_config
diffusion_config = config["diffusion_config"]  # basic hyperparameters

global diffusion_hyperparams
diffusion_hyperparams = calc_diffusion_hyperparams(
                      **diffusion_config)  # dictionary of all diffusion hyperparameters

global model_config

if train_config['use_model'] == 0:
    model_config = config['wavenet_config']
elif train_config['use_model'] == 1:
    model_config = config['sashimi_config']
elif train_config['use_model'] == 2:
    model_config = config['wavenet_config']
       
model_config['num_res_layers']=18           

In [5]:
output_directory = './results/mujoco/90'
ckpt_iter = 'max'
n_iters = 10000
iters_per_ckpt = 100
iters_per_logging = 100
# batch_size_per_gpu              
learning_rate = 0.0002
use_model = 2
only_generate_missing = 1       
masking = 'rm'               
missing_k = 20

In [6]:
local_path = "T{}_beta0{}_betaT{}".format(diffusion_config["T"],
                                              diffusion_config["beta_0"],
                                              diffusion_config["beta_T"])

output_directory = os.path.join(output_directory, local_path)
if not os.path.isdir(output_directory):
    os.makedirs(output_directory)
    os.chmod(output_directory, 0o775)               
print("output directory", output_directory, flush=True)

# map diffusion hyperparameters to gpu
for key in diffusion_hyperparams:
    if key != "T":
        diffusion_hyperparams[key] = diffusion_hyperparams[key].cuda()
        
        
# predefine model
if use_model == 0:
    net = DiffWaveImputer(**model_config).cuda()
elif use_model == 1:
    net = SSSDSAImputer(**model_config).cuda()
elif use_model == 2:
    net = SSSDS4Imputer(**model_config).cuda()    
else:
    print('Model chosen not available.')
    
print_size(net)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

output directory ./results/mujoco/90\T200_beta00.0001_betaT0.02
SSSDS4Imputer Parameters: 7.547142M


In [7]:
training_data = np.load(trainset_config['train_data_path'])
np.random.shuffle(training_data)
print(training_data.shape)

(5775, 30, 6)


In [8]:
training_data = np.split(training_data, 55, 0)      
print(len(training_data), training_data[0].shape)

training_data = np.array(training_data)
training_data = torch.from_numpy(training_data).float().cuda()
print(training_data.shape)


55 (105, 30, 6)
torch.Size([55, 105, 30, 6])


In [ ]:
missing_k = 20                    
iters = 0
loss_list=[]

while iters < n_iters + 1:

    for batch in training_data:             
        transposed_mask = get_mask_rm(batch[0], missing_k)     
        mask = transposed_mask.permute(1, 0)    
        mask = mask.repeat(batch.size()[0], 1, 1).float().cuda()  
        loss_mask = ~mask.bool()        
        batch = batch.permute(0, 2, 1)   
        optimizer.zero_grad()
        X = batch, batch, mask, loss_mask    
        loss = training_loss(net, nn.MSELoss(), X, diffusion_hyperparams,
                             only_generate_missing=only_generate_missing)

        loss.backward()
        optimizer.step()
           
        if iters % iters_per_logging == 0:
                print("iteration: {} \tloss: {}".format(iters, loss.item()))
                
        loss_list.append(loss.item())
        
        iters += 1
  

In [11]:
torch.save( net.state_dict(),"./sp500_S4_iter_10000.pth" )